In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
import pandas as pd

In [2]:
train_set=pd.read_pickle('dataset_next_state/integrators_next_state_train.pickle')
test_set=pd.read_pickle('dataset_next_state/integrators_next_state_test.pickle')

In [3]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)
        self.fc = nn.Linear(output_size, output_size)

    def forward(self, input, hidden):
        #print(f"input is {input}")
        #print(f"hidden is {hidden}")
        combined = torch.cat((input, hidden))
        #print(f"combined is {combined}")
        hidden = self.i2h(combined)
        #print(f"now hidden is {hidden}")
        output = self.h2o(hidden)
        output = self.fc(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(self.hidden_size)

In [4]:
n_iters = 1000
print_every = 1
learning_rate = 0.00005 # If you set this too high, it might explode. If too low, it might not learn
loss_function = nn.MSELoss()
n_hidden = 10
dimension=train_set[0][0].size(0)
rnn = RNN(dimension,n_hidden,dimension)
optimizer=optim.SGD(rnn.parameters(),learning_rate)

In [5]:
losses=[]
count=0
for iter in range(1, n_iters + 1):
    for trace in train_set:
        hidden=rnn.initHidden()
        for i in range(len(trace)-1):
            #print(hidden)
            output, hidden=rnn(trace[i],hidden)
            hidden=hidden.clone().detach()
            loss=loss_function(output,trace[i+1])
            
            optimizer.zero_grad()
            loss.backward()
            
            optimizer.step()

            losses.append(loss)
            if count % print_every == 0:
                print(loss)
            count+=1

tensor(1693.9185, grad_fn=<MseLossBackward0>)
tensor(1600.7054, grad_fn=<MseLossBackward0>)
tensor(1555.5464, grad_fn=<MseLossBackward0>)
tensor(1601.3883, grad_fn=<MseLossBackward0>)
tensor(1522.3390, grad_fn=<MseLossBackward0>)
tensor(1397.5331, grad_fn=<MseLossBackward0>)
tensor(1248.5742, grad_fn=<MseLossBackward0>)
tensor(1079.9159, grad_fn=<MseLossBackward0>)
tensor(884.4428, grad_fn=<MseLossBackward0>)
tensor(643.6159, grad_fn=<MseLossBackward0>)
tensor(458.1203, grad_fn=<MseLossBackward0>)
tensor(300.5465, grad_fn=<MseLossBackward0>)
tensor(189.2086, grad_fn=<MseLossBackward0>)
tensor(117.0858, grad_fn=<MseLossBackward0>)
tensor(73.3766, grad_fn=<MseLossBackward0>)
tensor(268.3183, grad_fn=<MseLossBackward0>)
tensor(161.5455, grad_fn=<MseLossBackward0>)
tensor(38.8501, grad_fn=<MseLossBackward0>)
tensor(56.1546, grad_fn=<MseLossBackward0>)
tensor(19.0144, grad_fn=<MseLossBackward0>)
tensor(7.1286, grad_fn=<MseLossBackward0>)
tensor(3.3019, grad_fn=<MseLossBackward0>)
tensor(1.9

KeyboardInterrupt: 

In [ ]:
def train(step,hidden,result,nn,learning_rate,loss_function):
    nn.zero_grad()
    output, hidden = nn(step, hidden)
    loss = loss_function(output, result)
    loss.backward(retain_graph=True)
    for p in nn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)
    return hidden,loss.item()


losses=[]
count=0
for iter in range(1, n_iters + 1):
    for trace in train_set:
        hidden=rnn.initHidden()
        for i in range(len(trace)-1):           
            hidden,loss = train(trace[i],hidden,trace[i+1],rnn,learning_rate,loss_function)
            losses.append(loss)
        if count % print_every == 0:
            print(loss)
        count+=1